# Imports

In [ ]:
import copy
import random
from datetime import datetime, timedelta, timezone
from importlib.resources import files
# from shodan.shodan.shodan import Shodan
from pprint import pprint, pformat
from time import time

from ledhntr import LEDHNTR
from ledhntr.plugins import HNTRPlugin
from ledhntr.data_classes import(
    Attribute, Entity, Relation, Thing
)
from ledhntr.helpers import dumps

led = LEDHNTR()
# shodan: Shodan
shodan = led.plugins['shodan']
censys = led.plugins['censys']
zeta = led.plugins['zeta']


def simple_search(
    plugin: HNTRPlugin = None,
    query: str = "",
    endpoint: str = "",
)->dict:

    plugin_name = plugin.__class__.__name__.lower()
    now = datetime.now(timezone.utc)
    # hosts_search

    res = plugin.search(simple={'endpoint': endpoint, 'query':query})
    data = dumps(res['raw'], compactly=True)
    date = f"{now.year}{now.month}{now.day}T{now.hour}{now.minute}{now.second}"
    with open(
        f'./data/{plugin_name}-{date}-{endpoint}.json', 
        'w'
    ) as f:
        f.write(data)
    return res

# Shodan

In [ ]:
now = datetime.now(timezone.utc)
plugin = shodan
plugin_name = plugin.__class__.__name__.lower()

# hosts_search
q = '''hash:-1587139099 ssl:"CN=ocserv VPN, O=ocserv"'''
endpoint = 'hosts_search'
res = simple_search(plugin, q, endpoint)

# host_details
query_number = 1
ips = []

for thing in res['things']:
    if thing.label=='ip':
        ips.append(thing.keyval)
        if len(ips) >= query_number:
            break

endpoint = 'host_details'
for ip in ips:
    res = simple_search(plugin, ip, endpoint)



# Censys

In [ ]:
now = datetime.now(timezone.utc)
plugin = censys
plugin_name = plugin.__class__.__name__.lower()

# hosts_search
q = '''same_service(
    services.tls.certificates.leaf_data.subject_dn="CN=ocserv VPN, O=ocserv" 
    AND services.banner_hashes="sha256:2206fcd3ea22d4dcf52f4dd9291753b80e0533e93586597e3c78b417a1537290"
)'''
endpoint = 'search'
res1 = simple_search(plugin, q, endpoint)

# host_details
query_number = 1
ips = []

for thing in res1['things']:
    if thing.label=='ip':
        ips.append(thing.keyval)
        if len(ips) >= query_number:
            break

endpoint = 'host_details'
for ip in ips:
    res2 = simple_search(plugin, ip, endpoint)



# Zeta

In [ ]:
now = datetime.now(timezone.utc)
plugin = zeta
plugin_name = plugin.__class__.__name__.lower()
domain = "poetpages.com"
ip = "65.21.51.58"

# hostname
endpoint = 'hostname'
zreshostname = simple_search(plugin, domain, endpoint)

# ip
endpoint = "ip"
zresip = simple_search(plugin, ip, endpoint)

# domain2d8s
endpoint = "domain2d8s"
zresd8s = simple_search(plugin, domain, endpoint)

# livedns
endpoint = "livedns"
zresdns = simple_search(plugin, domain, endpoint)